## 이전 단계 모델링

- [이전 단계 모델링](https://colab.research.google.com/drive/1wwJPrWH88Qfr8pH3wT5BX0rHtilOLyuv#scrollTo=4NViOy39tUTj)

# 1. 원데이터 및 추가 데이터 학습

## 1-1. 데이터 준비

In [ ]:
!pip install catboost

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression, Ridge
import optuna
import lightgbm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train.shape, test.shape, submission.shape, extra_data.shape

((42100, 19), (28068, 18), (28068, 2), (18137, 18))

In [ ]:
train.drop( "id", axis = 1, inplace = True )
test.drop( "id", axis = 1, inplace = True )

## 1-2. 최적화 파라미터 찾기

### 모듈 다운

In [ ]:
!pip install h2o

In [ ]:
import h2o
from h2o.automl import H2OAutoML
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator

In [ ]:
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from scipy.optimize import minimize
import numpy as np

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train.shape, test.shape, submission.shape, extra_data.shape

((42100, 19), (28068, 18), (28068, 2), (18137, 18))

In [ ]:
# H2O 클러스터 초기화
h2o.init( )

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 hours 46 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 4 days
H2O_cluster_name:,H2O_from_python_unknownUser_sv9cph
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.738 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
# CSV 파일을 H2OFrame으로 로드
train_h2o = h2o.H2OFrame( train )
test_h2o = h2o.H2OFrame( test )

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### 최적화 파라미터

In [ ]:
# 목표 변수 지정
y = 'booking_status'

In [ ]:
# AutoML이 600초 동안 다양한 모델을 훈련하고 평가하여 최적의 모델을 선택하고,
# 이 모델을 사용하여 테스트 데이터에 대한 예측을 생성

# AutoML 설정 및 실행
aml = H2OAutoML( max_runtime_secs = 600, seed = 42 )
aml.train( y = y, training_frame = train_h2o )

# AutoML 결과 확인
lb = aml.leaderboard
print( lb )

# 최고 성능 모델 선택
best_model = aml.leader

# 최고 성능 모델 세부 정보 출력
print( best_model )

# 테스트 데이터에 대한 예측
preds = best_model.predict( test_h2o )

# 예측 결과 확인
print( preds )

AutoML progress: |
04:37:20.383: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

█████
04:37:56.425: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.


04:38:00.850: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

██████
04:38:54.911: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
04:38:55.558: _response param, We ha

In [ ]:
# 랜덤포레스트 모델 정의
rf_model = H2ORandomForestEstimator( seed = 42 )
# 랜덤 포레스트 모델 학습
rf_model.train( x = train_h2o.columns,
                y = 'booking_status',
                training_frame = train_h2o )

# 테스트 데이터 예측
predictions = rf_model.predict( test_h2o )

# 예측 결과 출력
print( predictions )

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
  predict
0.102222
0.0602817
0.549048
0.0507857
0.431346
0.961292
0.429251
0.943333
0.04
0.50766
[28068 rows x 1 column]



In [ ]:
rf_model.params
# 0.961292

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'DRF_model_python_1713399505571_100',
   'type': 'Key<Model>',
   'URL': '/3/Models/DRF_model_python_1713399505571_100'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'Key_Frame__upload_b0280283fcaebb2a7988d7c60cd2464a.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/Key_Frame__upload_b0280283fcaebb2a7988d7c60cd2464a.hex'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'Key_Frame__upload_b0280283fcaebb2a7988d7c60cd2464a.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/Key_Frame__upload_b0280283fcaebb2a7988d7c60cd2464a.hex'}},
 'validation_frame': {'default': None, 'actual': None, 'input': None},
 'nfolds': {'default': 0, 'a

In [ ]:
from h2o.automl import H2OAutoML
from h2o.estimators import H2OStackedEnsembleEstimator

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )

## 1-3. 학습 준비

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train_targets = train[ "booking_status" ]
extra_targets = extra_data[ "booking_status" ]

train.drop(      "booking_status", axis = 1, inplace = True )
extra_data.drop( "booking_status", axis = 1, inplace = True )

In [ ]:
def objective( trial ):
    params = {
        # 트리 개수 지정
        'n_estimators' : 2000,
        # 트리 최대 깊이
        'depth' : trial.suggest_int( 'depth', 2, 15 ),
        # 트리의 학습에 사용되는 하이퍼파라미터
        'subsample' : trial.suggest_float( "subsample", 0.1, 1 ),
        'l2_leaf_reg' : trial.suggest_float( "l2_leaf_reg", 0.0, 5.0 ),
        'learning_rate' : trial.suggest_float( "learning_rate", 0.0001, 0.5 ),
        # 조기 정지를 위한 라운드 수
        'early_stopping_rounds' : 100
    }

    # 모델 초기화
    model = CatBoostClassifier( **params )

    model.fit( X_train, y_train,
               eval_set = [ ( X_valid, y_valid ) ],
               verbose = 0 )

    # 검증 데이터 ROC AUC 점수
    return roc_auc_score( y_valid, model.predict_proba( X_valid )[ : , 1 ] )

In [ ]:
# 테스트 데이터의 예측 결과를 저장할 빈 배열 생성
test_preds = np.zeros( len( test ) )
# 교차 검증 점수를 저장할 변수를 초기화
cv = 0
# 10-Fold 교차 검증을 수행하기 위한 StratifiedKFold 객체를 생성
kf = StratifiedKFold( n_splits = 10, shuffle = True, random_state = 42 )

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )

## 1-4. 모델 학습
- random depth 1 + catboost depth 3

In [ ]:
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from scipy.optimize import minimize
import numpy as np

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train.shape, test.shape, submission.shape, extra_data.shape

((42100, 19), (28068, 18), (28068, 2), (18137, 18))

In [ ]:
train.drop( "id", axis = 1, inplace = True )
test.drop( "id", axis = 1, inplace = True )

In [ ]:
train_targets = train[ "booking_status" ]
extra_targets = extra_data[ "booking_status" ]

train.drop(      "booking_status", axis = 1, inplace = True )
extra_data.drop( "booking_status", axis = 1, inplace = True )

In [ ]:
# 각 폴드에 대해 반복
for fold, ( train_idx, valid_idx ) in enumerate( kf.split( train, train_targets ) ):

    #  훈련 데이터와 검증 데이터를 나눔
    X_train, X_valid = train.iloc[ train_idx ], train.iloc[ valid_idx ]
    # 훈련 타겟과 검증 타겟을 나눔
    y_train, y_valid = train_targets.iloc[ train_idx ], train_targets.iloc[ valid_idx ]

    # 추가 데이터를 기존 훈련 데이터에 추가
    X_train = pd.concat( ( X_train, extra_data ) )
    y_train = pd.concat( ( y_train, extra_targets ) )

    # 결측 값 처리를 위한 SimpleImputer
    imputer = SimpleImputer( strategy = 'mean' )
    X_train_imputed = imputer.fit_transform( X_train )
    X_valid_imputed = imputer.fit_transform( X_valid )

    # 랜덤포레스트 모델 정의
    # 랜덤 포레스트 모델 객체 생성
    rf_classifier = RandomForestClassifier( random_state = 42,
                                            n_estimators = 50,
                                            max_depth = 1 )


    # CatBoost 모델 정의
    ctb = CatBoostClassifier( n_estimators = 10000,  # 트리 수 설정
                              early_stopping_rounds = 100,  # 조기 정지 설정
                              **{ # 모델의 하이퍼파라미터 설정
                                  'depth': 3,
                                  'subsample': 0.9037951675853159,
                                  'l2_leaf_reg': 4.085968446512874,
                                  'learning_rate': 0.33153188532829714 } )
    '''
      < subsample >
        - 각 트리가 학습에 사용하는 샘플의 비율을 결정
        - 값이 1보다 작으면 부트스트랩 샘플링 사용
        - 이 값이 클수록 더 많은 다양성을 가진 트리를 생성하며,
          작을수록 더 안정적인 트리를 생성

      < l2_leaf_reg >
        -  L2 정규화의 강도를 결정
        - 과적합을 방지
        - 값이 클수록 정규화의 강도 큼

      < learning_rate >
        - 가중치 업데이트에 대한 학습률
        - 작은 학습률은 모델을 안정적으로 만들지만 오래 걸림
    '''

    # 모델 학습

    # 랜덤포레스트 모델 학습
    rf_classifier.fit( X_train_imputed, y_train )

    # CatBoost 모델 학습
    ctb.fit( X_train, y_train, eval_set = ( X_valid, y_valid ), verbose = 100 )
    # eval_set: 검증 세트를 지정하여 모델의 성능 평가

    # 검증 데이터에 대한 클래스 확률 예측
    rf_test_preds =  rf_classifier.predict( X_valid_imputed )
    ctb_preds = ctb.predict_proba( X_valid )[ : , 1 ]
    ctb_test_preds = ctb.predict_proba( test )[ : , 1 ]

    #  ROC AUC 점수를 계산하여 모델의 성능 평가
    print (roc_auc_score ( y_valid, ctb_preds ) )

    # 메타 트레이닝 및 테스트 예측을 위해
    # CatBoost 모델의 검증 및 테스트 예측을 리스트에 저장
    meta_train = [ ctb_preds, rf_test_preds ]
    meta_test = [ ctb_test_preds ]

    # 최적의 가중치를 찾기 위한 목적 함수를 정의
    def roc_auc( weights ):
        fpred = np.zeros( len( meta_train[ 0 ] ) )
        for i, pred in enumerate( meta_train ):
            fpred += weights[ i ] * pred

        return -roc_auc_score( y_valid, fpred )

    starting_values = [ 0.33 ]*len( meta_train )
    cons = ( { 'type' : 'eq', 'fun' : lambda w : 1 - sum( w ) } )
    bounds = [ ( -1, 1 ) ] * len( meta_train )
    # 최적의 가중치 탐색
    res = minimize( roc_auc,
                    starting_values,
                    method = 'Nelder-Mead',
                    bounds = bounds,
                    constraints = cons )

    print( -res [ "fun" ] )

    # 교차 검증 점수에 최적 가중치에 대한 목적 함수 값의 음수를 더함
    # 최적화 과정에서 최대값 찾음
    cv -= res[ "fun" ]

    # 최적 가중치를 사용하여 테스트 데이터에 대한 예측을 업데이트
    for i, pred in enumerate( meta_test ):
        test_preds += res[ "x" ][ i ] * pred

# 모든 폴드에 대한 교차 검증 점수의 평균을 출력
print( f"CV: { cv / ( fold + 1) }" )

0:	learn: 0.5801179	test: 0.5810034	best: 0.5810034 (0)	total: 11.1ms	remaining: 1m 51s
100:	learn: 0.3804657	test: 0.4125511	best: 0.4125511 (100)	total: 1.18s	remaining: 1m 56s
200:	learn: 0.3660269	test: 0.4050277	best: 0.4046275 (189)	total: 3.12s	remaining: 2m 32s
300:	learn: 0.3562293	test: 0.3991781	best: 0.3991586 (298)	total: 4.27s	remaining: 2m 17s
400:	learn: 0.3489805	test: 0.3957893	best: 0.3957416 (399)	total: 6.11s	remaining: 2m 26s
500:	learn: 0.3432186	test: 0.3936444	best: 0.3933639 (494)	total: 8.56s	remaining: 2m 42s
600:	learn: 0.3384171	test: 0.3914283	best: 0.3913247 (597)	total: 9.92s	remaining: 2m 35s
700:	learn: 0.3339469	test: 0.3898378	best: 0.3898378 (700)	total: 11.1s	remaining: 2m 27s
800:	learn: 0.3299965	test: 0.3913249	best: 0.3896508 (713)	total: 12.2s	remaining: 2m 20s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.389650813
bestIteration = 713

Shrink model to first 714 iterations.
0.9013250473484848
0.9013250473484848


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0:	learn: 0.5806034	test: 0.5757682	best: 0.5757682 (0)	total: 11ms	remaining: 1m 49s
100:	learn: 0.3811323	test: 0.4130820	best: 0.4130820 (100)	total: 1.13s	remaining: 1m 50s
200:	learn: 0.3661259	test: 0.4060067	best: 0.4060067 (200)	total: 2.23s	remaining: 1m 48s
300:	learn: 0.3560205	test: 0.4009237	best: 0.4007827 (290)	total: 3.34s	remaining: 1m 47s
400:	learn: 0.3492666	test: 0.3996628	best: 0.3995570 (377)	total: 4.5s	remaining: 1m 47s
500:	learn: 0.3424686	test: 0.3978555	best: 0.3974233 (470)	total: 5.66s	remaining: 1m 47s
600:	learn: 0.3375338	test: 0.3972527	best: 0.3966896 (559)	total: 8.22s	remaining: 2m 8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3966896099
bestIteration = 559

Shrink model to first 560 iterations.
0.8973243371212123
0.8973243371212123


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0:	learn: 0.5798108	test: 0.5830340	best: 0.5830340 (0)	total: 11.2ms	remaining: 1m 52s
100:	learn: 0.3804918	test: 0.4096709	best: 0.4096488 (99)	total: 1.16s	remaining: 1m 53s
200:	learn: 0.3653004	test: 0.4031296	best: 0.4031296 (200)	total: 2.28s	remaining: 1m 51s
300:	learn: 0.3549492	test: 0.3980596	best: 0.3977047 (293)	total: 3.41s	remaining: 1m 49s
400:	learn: 0.3483250	test: 0.3960509	best: 0.3960152 (386)	total: 4.54s	remaining: 1m 48s
500:	learn: 0.3425244	test: 0.3937044	best: 0.3936864 (487)	total: 5.7s	remaining: 1m 48s
600:	learn: 0.3372616	test: 0.3929142	best: 0.3926140 (587)	total: 6.85s	remaining: 1m 47s
700:	learn: 0.3327411	test: 0.3912311	best: 0.3912311 (700)	total: 8s	remaining: 1m 46s
800:	learn: 0.3291883	test: 0.3905569	best: 0.3905569 (800)	total: 9.15s	remaining: 1m 45s
900:	learn: 0.3257437	test: 0.3900916	best: 0.3897632 (811)	total: 13.5s	remaining: 2m 16s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.389763244
bestIteration = 811

...

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.8913961223780205
0:	learn: 0.5807528	test: 0.5755939	best: 0.5755939 (0)	total: 10.7ms	remaining: 1m 46s
100:	learn: 0.3801834	test: 0.4079136	best: 0.4079136 (100)	total: 1.17s	remaining: 1m 54s
200:	learn: 0.3648400	test: 0.4038398	best: 0.4037286 (198)	total: 3.62s	remaining: 2m 56s
300:	learn: 0.3551966	test: 0.3986481	best: 0.3986481 (300)	total: 5.75s	remaining: 3m 5s
400:	learn: 0.3483355	test: 0.3968436	best: 0.3965241 (396)	total: 6.87s	remaining: 2m 44s
500:	learn: 0.3421361	test: 0.3945097	best: 0.3945097 (500)	total: 8.02s	remaining: 2m 32s
600:	learn: 0.3372195	test: 0.3935371	best: 0.3930911 (581)	total: 9.16s	remaining: 2m 23s
700:	learn: 0.3327898	test: 0.3930979	best: 0.3927904 (671)	total: 10.4s	remaining: 2m 17s
800:	learn: 0.3291309	test: 0.3919831	best: 0.3919271 (797)	total: 11.5s	remaining: 2m 12s
900:	learn: 0.3255423	test: 0.3923780	best: 0.3916010 (858)	total: 12.7s	remaining: 2m 7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3916010

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0:	learn: 0.5802944	test: 0.5791632	best: 0.5791632 (0)	total: 10.8ms	remaining: 1m 48s
100:	learn: 0.3801576	test: 0.4061715	best: 0.4060381 (99)	total: 1.12s	remaining: 1m 50s
200:	learn: 0.3649032	test: 0.3984154	best: 0.3982627 (196)	total: 3.67s	remaining: 2m 59s
300:	learn: 0.3554964	test: 0.3948986	best: 0.3948986 (300)	total: 5.57s	remaining: 2m 59s
400:	learn: 0.3487211	test: 0.3928082	best: 0.3928082 (400)	total: 6.7s	remaining: 2m 40s
500:	learn: 0.3428346	test: 0.3908257	best: 0.3906399 (496)	total: 7.86s	remaining: 2m 28s
600:	learn: 0.3377649	test: 0.3894282	best: 0.3890255 (581)	total: 8.96s	remaining: 2m 20s
700:	learn: 0.3334689	test: 0.3877795	best: 0.3877062 (685)	total: 10.1s	remaining: 2m 13s
800:	learn: 0.3298223	test: 0.3874942	best: 0.3872385 (768)	total: 11.2s	remaining: 2m 8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3872385245
bestIteration = 768

Shrink model to first 769 iterations.
0.9012145587041047
0.9012145587041047
CV: 17.047

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


In [ ]:
train = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )

In [ ]:
# test_preds는 이전 단계에서 각 폴드에서 얻은 예측 결과
# (fold+1)은 폴드의 수로 나누어 결과를 평균화
submission[ "booking_status" ] = test_preds / ( fold + 1 )
y = 'booking_status'

# 테스트 데이터에서 'id' 열을 제외한 중복된 특성 가져옴
dup_features = test.drop( columns = 'id').columns.tolist( )

# 이너 조인하여 중복된 샘플을 찾음
# 데이터프레임 생성
values_to_assign = test.merge( train.drop( columns = 'id' ), on = dup_features, how = 'inner' )[[ 'id', y ] ]

# 중복된 샘플의 예측 값을 수정하기 위해 사용할 매핑 딕셔너리를 생성
# booking_status 최대값 : 0, 최소값 : 1 매핑
map_di = { 0 : submission[ y ].max( ), 1 : submission[ y ].min( ) }
# 중복된 샘플의 booking_status 값 수정
# 매핑 딕셔너리를 사용해 수정된 값 할당
submission.loc[ submission.id.isin( values_to_assign.id ), y ] = values_to_assign[ y ].map( map_di ).values

In [ ]:
submission_12 = submission
submission_12

,id,booking_status
0,42100,1.015627
1,42101,0.287843
2,42102,2.381864
3,42103,0.344884
4,42104,2.618794
...,...,...
28063,70163,6.906082
28064,70164,0.222092
28065,70165,0.544543
28066,70166,3.318239


In [ ]:
submission_12.to_csv( "submission_12.csv", index = False )

# 2. 살린 중복 데이터 및 추가 데이터 학습

## 2-1. 중복데이터 변경

[중복데이터 전처리](https://colab.research.google.com/drive/10QxT7BN3hAJssFy0e6URItHOrO1uC-J2?usp=sharing)

### 2-1-1. 데이터 준비

In [ ]:
!pip install catboost

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression, Ridge
import optuna
import lightgbm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

### 2-1-2. 최적화 파라미터 찾기

#### 모듈 다운

In [ ]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.6/265.6 MB 2.6 MB/s eta 0:00:00


In [ ]:
import h2o
from h2o.automl import H2OAutoML
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator

In [ ]:
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from scipy.optimize import minimize
import numpy as np

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/change_train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/0.5test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train.shape, test.shape, submission.shape, extra_data.shape

((59521, 19), (28068, 19), (28068, 2), (18137, 18))

In [ ]:
# H2O 클러스터 초기화
h2o.init( )

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpn9hirryi
  JVM stdout: /tmp/tmpn9hirryi/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpn9hirryi/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_unknownUser_349mga
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
# CSV 파일을 H2OFrame으로 로드
train_h2o = h2o.H2OFrame( train )
test_h2o = h2o.H2OFrame( test )

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


#### 최적화 파라미터

In [ ]:
# 목표 변수 지정
y = 'booking_status'

In [ ]:
# AutoML이 600초 동안 다양한 모델을 훈련하고 평가하여 최적의 모델을 선택하고,
# 이 모델을 사용하여 테스트 데이터에 대한 예측을 생성

# AutoML 설정 및 실행
aml = H2OAutoML( max_runtime_secs = 600, seed = 42 )
aml.train( y = y, training_frame = train_h2o )

# AutoML 결과 확인
lb = aml.leaderboard
print( lb )

# 최고 성능 모델 선택
best_model = aml.leader

# 최고 성능 모델 세부 정보 출력
print( best_model )

# 테스트 데이터에 대한 예측
preds = best_model.predict( test_h2o )

# 예측 결과 확인
print( preds )

AutoML progress: |
01:49:59.688: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

███████████
01:51:35.490: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

█
01:51:41.957: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

███████████
01:53:28.187: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
01:53:29.779: _response 

In [ ]:
# 랜덤포레스트 모델 정의
rf_model = H2ORandomForestEstimator( seed = 42 )
# 랜덤 포레스트 모델 학습
rf_model.train( x = train_h2o.columns,
                y = 'booking_status',
                training_frame = train_h2o )

# 테스트 데이터 예측
predictions = rf_model.predict( test_h2o )

# 예측 결과 출력
print( predictions )

drf Model Build progress: |

/usr/local/lib/python3.10/dist-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
   predict
0.06
0.0195511
0.431139
0.00289188
0.364204
0.905036
0.47099
0.961847
0
0.591753
[28068 rows x 1 column]



In [ ]:
rf_model.params
# 0.961847

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'DRF_model_python_1713750574073_22',
   'type': 'Key<Model>',
   'URL': '/3/Models/DRF_model_python_1713750574073_22'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'Key_Frame__upload_81b3350be7e48817ea89128c55364dbe.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/Key_Frame__upload_81b3350be7e48817ea89128c55364dbe.hex'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'Key_Frame__upload_81b3350be7e48817ea89128c55364dbe.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/Key_Frame__upload_81b3350be7e48817ea89128c55364dbe.hex'}},
 'validation_frame': {'default': None, 'actual': None, 'input': None},
 'nfolds': {'default': 0, 'act

### 2-1-3. 학습 준비

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
def objective( trial ):
    params = {
        # 트리 개수 지정
        'n_estimators' : 2000,
        # 트리 최대 깊이
        'depth' : trial.suggest_int( 'depth', 2, 15 ),
        # 트리의 학습에 사용되는 하이퍼파라미터
        'subsample' : trial.suggest_float( "subsample", 0.1, 1 ),
        'l2_leaf_reg' : trial.suggest_float( "l2_leaf_reg", 0.0, 5.0 ),
        'learning_rate' : trial.suggest_float( "learning_rate", 0.0001, 0.5 ),
        # 조기 정지를 위한 라운드 수
        'early_stopping_rounds' : 100
    }

    # 모델 초기화
    model = CatBoostClassifier( **params )

    model.fit( X_train, y_train,
               eval_set = [ ( X_valid, y_valid ) ],
               verbose = 0 )

    # 검증 데이터 ROC AUC 점수
    return roc_auc_score( y_valid, model.predict_proba( X_valid )[ : , 1 ] )

In [ ]:
# 테스트 데이터의 예측 결과를 저장할 빈 배열 생성
test_preds = np.zeros( len( test ) )
# 교차 검증 점수를 저장할 변수를 초기화
cv = 0
# 10-Fold 교차 검증을 수행하기 위한 StratifiedKFold 객체를 생성
kf = StratifiedKFold( n_splits = 10, shuffle = True, random_state = 42 )

### 2-1-4. 모델 학습
- random depth 2 + catboost depth 2

In [ ]:
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from scipy.optimize import minimize
import numpy as np

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/change_train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/0.5test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train.shape, test.shape, submission.shape, extra_data.shape

((59521, 19), (28068, 19), (28068, 2), (18137, 18))

In [ ]:
# train.drop( "id", axis = 1, inplace = True )
test.drop( "id", axis = 1, inplace = True )

In [ ]:
train_targets = train[ "booking_status" ]
extra_targets = extra_data[ "booking_status" ]

train.drop(      "booking_status", axis = 1, inplace = True )
extra_data.drop( "booking_status", axis = 1, inplace = True )

In [ ]:
# 각 폴드에 대해 반복
for fold, ( train_idx, valid_idx ) in enumerate( kf.split( train, train_targets ) ):

    #  훈련 데이터와 검증 데이터를 나눔
    X_train, X_valid = train.iloc[ train_idx ], train.iloc[ valid_idx ]
    # 훈련 타겟과 검증 타겟을 나눔
    y_train, y_valid = train_targets.iloc[ train_idx ], train_targets.iloc[ valid_idx ]

    # 추가 데이터를 기존 훈련 데이터에 추가
    X_train = pd.concat( ( X_train, extra_data ) )
    y_train = pd.concat( ( y_train, extra_targets ) )

    # 결측 값 처리를 위한 SimpleImputer
    imputer = SimpleImputer( strategy = 'mean' )
    X_train_imputed = imputer.fit_transform( X_train )
    X_valid_imputed = imputer.fit_transform( X_valid )

    # 랜덤포레스트 모델 정의
    # 랜덤 포레스트 모델 객체 생성
    rf_classifier = RandomForestClassifier( random_state = 42,
                                            n_estimators = 50,
                                            max_depth = 2 )


    # CatBoost 모델 정의
    ctb = CatBoostClassifier( n_estimators = 10000,  # 트리 수 설정
                              early_stopping_rounds = 100,  # 조기 정지 설정
                              **{ # 모델의 하이퍼파라미터 설정
                                  'depth': 2,
                                  'subsample': 0.9037951675853159,
                                  'l2_leaf_reg': 4.085968446512874,
                                  'learning_rate': 0.33153188532829714 } )
    '''
      < subsample >
        - 각 트리가 학습에 사용하는 샘플의 비율을 결정
        - 값이 1보다 작으면 부트스트랩 샘플링 사용
        - 이 값이 클수록 더 많은 다양성을 가진 트리를 생성하며,
          작을수록 더 안정적인 트리를 생성

      < l2_leaf_reg >
        -  L2 정규화의 강도를 결정
        - 과적합을 방지
        - 값이 클수록 정규화의 강도 큼

      < learning_rate >
        - 가중치 업데이트에 대한 학습률
        - 작은 학습률은 모델을 안정적으로 만들지만 오래 걸림
    '''

    # 모델 학습

    # 랜덤포레스트 모델 학습
    rf_classifier.fit( X_train_imputed, y_train )

    # CatBoost 모델 학습
    ctb.fit( X_train, y_train, eval_set = ( X_valid, y_valid ), verbose = 100 )
    # eval_set: 검증 세트를 지정하여 모델의 성능 평가

    # 검증 데이터에 대한 클래스 확률 예측
    rf_test_preds =  rf_classifier.predict( X_valid_imputed )
    ctb_preds = ctb.predict_proba( X_valid )[ : , 1 ]
    ctb_test_preds = ctb.predict_proba( test )[ : , 1 ]

    #  ROC AUC 점수를 계산하여 모델의 성능 평가
    print (roc_auc_score ( y_valid, ctb_preds ) )

    # 메타 트레이닝 및 테스트 예측을 위해
    # CatBoost 모델의 검증 및 테스트 예측을 리스트에 저장
    meta_train = [ ctb_preds, rf_test_preds ]
    meta_test = [ ctb_test_preds ]

    # 최적의 가중치를 찾기 위한 목적 함수를 정의
    def roc_auc( weights ):
        fpred = np.zeros( len( meta_train[ 0 ] ) )
        for i, pred in enumerate( meta_train ):
            fpred += weights[ i ] * pred

        return -roc_auc_score( y_valid, fpred )

    starting_values = [ 0.33 ]*len( meta_train )
    cons = ( { 'type' : 'eq', 'fun' : lambda w : 1 - sum( w ) } )
    bounds = [ ( -1, 1 ) ] * len( meta_train )
    # 최적의 가중치 탐색
    res = minimize( roc_auc,
                    starting_values,
                    method = 'Nelder-Mead',
                    bounds = bounds,
                    constraints = cons )

    print( -res [ "fun" ] )

    # 교차 검증 점수에 최적 가중치에 대한 목적 함수 값의 음수를 더함
    # 최적화 과정에서 최대값 찾음
    cv -= res[ "fun" ]

    # 최적 가중치를 사용하여 테스트 데이터에 대한 예측을 업데이트
    for i, pred in enumerate( meta_test ):
        test_preds += res[ "x" ][ i ] * pred

# 모든 폴드에 대한 교차 검증 점수의 평균을 출력
print( f"CV: { cv / ( fold + 1) }" )

0:	learn: 0.6078874	test: 0.6070942	best: 0.6070942 (0)	total: 12.4ms	remaining: 2m 4s
100:	learn: 0.3816886	test: 0.4046128	best: 0.4046128 (100)	total: 1.45s	remaining: 2m 21s
200:	learn: 0.3638173	test: 0.3918075	best: 0.3918075 (200)	total: 2.89s	remaining: 2m 20s
300:	learn: 0.3541776	test: 0.3850182	best: 0.3848994 (297)	total: 4.21s	remaining: 2m 15s
400:	learn: 0.3480116	test: 0.3813263	best: 0.3813263 (400)	total: 5.51s	remaining: 2m 11s
500:	learn: 0.3428953	test: 0.3780929	best: 0.3780748 (499)	total: 6.8s	remaining: 2m 8s
600:	learn: 0.3389661	test: 0.3750980	best: 0.3750614 (599)	total: 8.99s	remaining: 2m 20s
700:	learn: 0.3356809	test: 0.3733573	best: 0.3733255 (693)	total: 11.6s	remaining: 2m 33s
800:	learn: 0.3332949	test: 0.3717615	best: 0.3717615 (800)	total: 12.9s	remaining: 2m 27s
900:	learn: 0.3307558	test: 0.3703987	best: 0.3703987 (900)	total: 14.1s	remaining: 2m 22s
1000:	learn: 0.3285485	test: 0.3692866	best: 0.3692238 (998)	total: 15.5s	remaining: 2m 18s
1100

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9127187699541418
0:	learn: 0.6076713	test: 0.6110307	best: 0.6110307 (0)	total: 13.6ms	remaining: 2m 15s
100:	learn: 0.3816859	test: 0.3955584	best: 0.3955153 (99)	total: 1.34s	remaining: 2m 11s
200:	learn: 0.3650271	test: 0.3821652	best: 0.3821652 (200)	total: 2.64s	remaining: 2m 8s
300:	learn: 0.3557448	test: 0.3757298	best: 0.3756623 (297)	total: 3.95s	remaining: 2m 7s
400:	learn: 0.3489385	test: 0.3712622	best: 0.3712622 (400)	total: 5.25s	remaining: 2m 5s
500:	learn: 0.3441491	test: 0.3679022	best: 0.3679022 (500)	total: 6.57s	remaining: 2m 4s
600:	learn: 0.3400439	test: 0.3654288	best: 0.3654288 (600)	total: 8.64s	remaining: 2m 15s
700:	learn: 0.3368538	test: 0.3634148	best: 0.3633857 (694)	total: 11.4s	remaining: 2m 30s
800:	learn: 0.3338893	test: 0.3621768	best: 0.3621363 (797)	total: 12.7s	remaining: 2m 25s
900:	learn: 0.3316621	test: 0.3611339	best: 0.3611339 (900)	total: 14s	remaining: 2m 21s
1000:	learn: 0.3296602	test: 0.3598256	best: 0.3597065 (991)	total: 15.3s	remaini

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9165986607728899
0:	learn: 0.6080919	test: 0.6108466	best: 0.6108466 (0)	total: 12.3ms	remaining: 2m 3s
100:	learn: 0.3807606	test: 0.3921436	best: 0.3921436 (100)	total: 1.34s	remaining: 2m 11s
200:	learn: 0.3634496	test: 0.3787049	best: 0.3787049 (200)	total: 2.62s	remaining: 2m 7s
300:	learn: 0.3540696	test: 0.3710741	best: 0.3710741 (300)	total: 3.9s	remaining: 2m 5s
400:	learn: 0.3476491	test: 0.3664312	best: 0.3664312 (400)	total: 5.18s	remaining: 2m 4s
500:	learn: 0.3430096	test: 0.3636651	best: 0.3636431 (493)	total: 6.5s	remaining: 2m 3s
600:	learn: 0.3391029	test: 0.3617352	best: 0.3615906 (597)	total: 7.81s	remaining: 2m 2s
700:	learn: 0.3360762	test: 0.3598932	best: 0.3598932 (700)	total: 10s	remaining: 2m 12s
800:	learn: 0.3334480	test: 0.3588843	best: 0.3588506 (793)	total: 12.6s	remaining: 2m 25s
900:	learn: 0.3310646	test: 0.3572393	best: 0.3572179 (898)	total: 13.9s	remaining: 2m 20s
1000:	learn: 0.3290106	test: 0.3564855	best: 0.3564855 (1000)	total: 15.2s	remaining

...

0.9143393318198005
0:	learn: 0.6069429	test: 0.6084032	best: 0.6084032 (0)	total: 16.7ms	remaining: 2m 47s
100:	learn: 0.3830813	test: 0.3856452	best: 0.3856452 (100)	total: 2.47s	remaining: 4m 1s
200:	learn: 0.3659891	test: 0.3704502	best: 0.3704502 (200)	total: 4.87s	remaining: 3m 57s
300:	learn: 0.3572972	test: 0.3641270	best: 0.3641270 (300)	total: 6.2s	remaining: 3m 19s
400:	learn: 0.3506438	test: 0.3580853	best: 0.3580853 (400)	total: 7.51s	remaining: 2m 59s
500:	learn: 0.3454709	test: 0.3539226	best: 0.3539226 (500)	total: 8.79s	remaining: 2m 46s
600:	learn: 0.3410046	test: 0.3506624	best: 0.3506259 (597)	total: 10.1s	remaining: 2m 37s
700:	learn: 0.3375909	test: 0.3484937	best: 0.3484927 (698)	total: 11.4s	remaining: 2m 31s
800:	learn: 0.3351268	test: 0.3467782	best: 0.3467504 (786)	total: 12.7s	remaining: 2m 25s
900:	learn: 0.3327260	test: 0.3451916	best: 0.3451418 (898)	total: 14s	remaining: 2m 21s
1000:	learn: 0.3306489	test: 0.3439883	best: 0.3439883 (1000)	total: 16.6s	rem

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9235017574012139
0:	learn: 0.6069962	test: 0.6071421	best: 0.6071421 (0)	total: 19.4ms	remaining: 3m 14s
100:	learn: 0.3834847	test: 0.3904767	best: 0.3904767 (100)	total: 1.36s	remaining: 2m 13s
200:	learn: 0.3661553	test: 0.3780686	best: 0.3780686 (200)	total: 2.66s	remaining: 2m 9s
300:	learn: 0.3566037	test: 0.3714860	best: 0.3714860 (300)	total: 4.89s	remaining: 2m 37s
400:	learn: 0.3496305	test: 0.3662048	best: 0.3662048 (400)	total: 7.47s	remaining: 2m 58s
500:	learn: 0.3449697	test: 0.3629454	best: 0.3629454 (500)	total: 8.78s	remaining: 2m 46s
600:	learn: 0.3412940	test: 0.3604843	best: 0.3604843 (600)	total: 10.1s	remaining: 2m 37s
700:	learn: 0.3377031	test: 0.3586933	best: 0.3585773 (668)	total: 11.4s	remaining: 2m 30s
800:	learn: 0.3347460	test: 0.3567159	best: 0.3566514 (792)	total: 12.7s	remaining: 2m 25s
900:	learn: 0.3319990	test: 0.3547217	best: 0.3546817 (899)	total: 14s	remaining: 2m 21s
1000:	learn: 0.3300276	test: 0.3540187	best: 0.3539369 (977)	total: 15.3s	rem

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9175045410821567
CV: 9.160594841122364


In [ ]:
train = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )

In [ ]:
# test_preds는 이전 단계에서 각 폴드에서 얻은 예측 결과
# (fold+1)은 폴드의 수로 나누어 결과를 평균화
submission[ "booking_status" ] = test_preds / ( fold + 1 )
y = 'booking_status'

# 테스트 데이터에서 'id' 열을 제외한 중복된 특성 가져옴
dup_features = test.drop( columns = 'id').columns.tolist( )

# 이너 조인하여 중복된 샘플을 찾음
# 데이터프레임 생성
values_to_assign = test.merge( train.drop( columns = 'id' ), on = dup_features, how = 'inner' )[[ 'id', y ] ]

# 중복된 샘플의 예측 값을 수정하기 위해 사용할 매핑 딕셔너리를 생성
# booking_status 최대값 : 0, 최소값 : 1 매핑
map_di = { 0 : submission[ y ].max( ), 1 : submission[ y ].min( ) }
# 중복된 샘플의 booking_status 값 수정
# 매핑 딕셔너리를 사용해 수정된 값 할당
submission.loc[ submission.id.isin( values_to_assign.id ), y ] = values_to_assign[ y ].map( map_di ).values

In [ ]:
submission_1_2 = submission
submission_1_2

,id,booking_status
0,42100,0.765790
1,42101,0.294185
2,42102,1.680203
3,42103,0.186648
4,42104,1.800155
...,...,...
28063,70163,5.387051
28064,70164,0.122163
28065,70165,0.521459
28066,70166,1.951797


In [ ]:
submission_1_2.to_csv( "submission_1-2.csv", index = False )

# 3. 삭제 중복 데이터 및 추가 데이터 학습

[중복데이터 전처리](https://colab.research.google.com/drive/10QxT7BN3hAJssFy0e6URItHOrO1uC-J2?usp=sharing)

## 3-1. 데이터 준비

In [ ]:
!pip install catboost

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression, Ridge
import optuna
import lightgbm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/del_train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/0.5test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train.shape, test.shape, submission.shape, extra_data.shape

((53005, 19), (28068, 19), (28068, 2), (18137, 18))

## 3-2. 최적화 파라미터 찾기

### 모듈 다운

In [ ]:
!pip install h2o

In [ ]:
import h2o
from h2o.automl import H2OAutoML
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator

In [ ]:
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from scipy.optimize import minimize
import numpy as np

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/change_train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/0.5test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train.shape, test.shape, submission.shape, extra_data.shape

((59521, 19), (28068, 19), (28068, 2), (18137, 18))

In [ ]:
# H2O 클러스터 초기화
h2o.init( )

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,31 mins 16 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_unknownUser_349mga
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.919 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
# CSV 파일을 H2OFrame으로 로드
train_h2o = h2o.H2OFrame( train )
test_h2o = h2o.H2OFrame( test )

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### 최적화 파라미터

In [ ]:
# 목표 변수 지정
y = 'booking_status'

In [ ]:
# AutoML이 600초 동안 다양한 모델을 훈련하고 평가하여 최적의 모델을 선택하고,
# 이 모델을 사용하여 테스트 데이터에 대한 예측을 생성

# AutoML 설정 및 실행
aml = H2OAutoML( max_runtime_secs = 600, seed = 42 )
aml.train( y = y, training_frame = train_h2o )

# AutoML 결과 확인
lb = aml.leaderboard
print( lb )

# 최고 성능 모델 선택
best_model = aml.leader

# 최고 성능 모델 세부 정보 출력
print( best_model )

# 테스트 데이터에 대한 예측
preds = best_model.predict( test_h2o )

# 예측 결과 확인
print( preds )

AutoML progress: |
02:09:28.236: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

████████
02:10:38.145: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

█
02:10:45.737: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.

███████████████
02:13:12.250: _response param, We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.


02:13:13.89: _respons

In [ ]:
# 랜덤포레스트 모델 정의
rf_model = H2ORandomForestEstimator( seed = 42 )
# 랜덤 포레스트 모델 학습
rf_model.train( x = train_h2o.columns,
                y = 'booking_status',
                training_frame = train_h2o )

# 테스트 데이터 예측
predictions = rf_model.predict( test_h2o )

# 예측 결과 출력
print( predictions )

drf Model Build progress: |

/usr/local/lib/python3.10/dist-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
   predict
0.06
0.0195511
0.431139
0.00289188
0.364204
0.905036
0.47099
0.961847
0
0.591753
[28068 rows x 1 column]



In [ ]:
rf_model.params
# 0.961847

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'DRF_model_python_1713750574073_43',
   'type': 'Key<Model>',
   'URL': '/3/Models/DRF_model_python_1713750574073_43'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'Key_Frame__upload_8eacbae6bf6420af83c130f3a87d4373.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/Key_Frame__upload_8eacbae6bf6420af83c130f3a87d4373.hex'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'Key_Frame__upload_8eacbae6bf6420af83c130f3a87d4373.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/Key_Frame__upload_8eacbae6bf6420af83c130f3a87d4373.hex'}},
 'validation_frame': {'default': None, 'actual': None, 'input': None},
 'nfolds': {'default': 0, 'act

## 3-3. 학습 준비

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
def objective( trial ):
    params = {
        # 트리 개수 지정
        'n_estimators' : 2000,
        # 트리 최대 깊이
        'depth' : trial.suggest_int( 'depth', 2, 15 ),
        # 트리의 학습에 사용되는 하이퍼파라미터
        'subsample' : trial.suggest_float( "subsample", 0.1, 1 ),
        'l2_leaf_reg' : trial.suggest_float( "l2_leaf_reg", 0.0, 5.0 ),
        'learning_rate' : trial.suggest_float( "learning_rate", 0.0001, 0.5 ),
        # 조기 정지를 위한 라운드 수
        'early_stopping_rounds' : 100
    }

    # 모델 초기화
    model = CatBoostClassifier( **params )

    model.fit( X_train, y_train,
               eval_set = [ ( X_valid, y_valid ) ],
               verbose = 0 )

    # 검증 데이터 ROC AUC 점수
    return roc_auc_score( y_valid, model.predict_proba( X_valid )[ : , 1 ] )

In [ ]:
# 테스트 데이터의 예측 결과를 저장할 빈 배열 생성
test_preds = np.zeros( len( test ) )
# 교차 검증 점수를 저장할 변수를 초기화
cv = 0
# 10-Fold 교차 검증을 수행하기 위한 StratifiedKFold 객체를 생성
kf = StratifiedKFold( n_splits = 10, shuffle = True, random_state = 42 )

## 3-4. 모델링 학습
- random depth 10 + catboost depth 8

In [ ]:
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from scipy.optimize import minimize
import numpy as np

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/del_train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/0.5test.csv' )

In [ ]:
# train.drop( "id", axis = 1, inplace = True )
test.drop( "id", axis = 1, inplace = True )

In [ ]:
train_targets = train[ "booking_status" ]
extra_targets = extra_data[ "booking_status" ]

train.drop(      "booking_status", axis = 1, inplace = True )
extra_data.drop( "booking_status", axis = 1, inplace = True )

In [ ]:
# 각 폴드에 대해 반복
for fold, ( train_idx, valid_idx ) in enumerate( kf.split( train, train_targets ) ):

    #  훈련 데이터와 검증 데이터를 나눔
    X_train, X_valid = train.iloc[ train_idx ], train.iloc[ valid_idx ]
    # 훈련 타겟과 검증 타겟을 나눔
    y_train, y_valid = train_targets.iloc[ train_idx ], train_targets.iloc[ valid_idx ]

    # 추가 데이터를 기존 훈련 데이터에 추가
    X_train = pd.concat( ( X_train, extra_data ) )
    y_train = pd.concat( ( y_train, extra_targets ) )

    # 결측 값 처리를 위한 SimpleImputer
    imputer = SimpleImputer( strategy = 'mean' )
    X_train_imputed = imputer.fit_transform( X_train )
    X_valid_imputed = imputer.fit_transform( X_valid )

    # 랜덤포레스트 모델 정의
    # 랜덤 포레스트 모델 객체 생성
    rf_classifier = RandomForestClassifier( random_state = 42,
                                            n_estimators = 50,
                                            max_depth = 10 )


    # CatBoost 모델 정의
    ctb = CatBoostClassifier( n_estimators = 10000,  # 트리 수 설정
                              early_stopping_rounds = 100,  # 조기 정지 설정
                              **{ # 모델의 하이퍼파라미터 설정
                                  'depth': 8,
                                  'subsample': 0.9037951675853159,
                                  'l2_leaf_reg': 4.085968446512874,
                                  'learning_rate': 0.33153188532829714 } )
    '''
      < subsample >
        - 각 트리가 학습에 사용하는 샘플의 비율을 결정
        - 값이 1보다 작으면 부트스트랩 샘플링 사용
        - 이 값이 클수록 더 많은 다양성을 가진 트리를 생성하며,
          작을수록 더 안정적인 트리를 생성

      < l2_leaf_reg >
        -  L2 정규화의 강도를 결정
        - 과적합을 방지
        - 값이 클수록 정규화의 강도 큼

      < learning_rate >
        - 가중치 업데이트에 대한 학습률
        - 작은 학습률은 모델을 안정적으로 만들지만 오래 걸림
    '''

    # 모델 학습

    # 랜덤포레스트 모델 학습
    rf_classifier.fit( X_train_imputed, y_train )

    # CatBoost 모델 학습
    ctb.fit( X_train, y_train, eval_set = ( X_valid, y_valid ), verbose = 100 )
    # eval_set: 검증 세트를 지정하여 모델의 성능 평가

    # 검증 데이터에 대한 클래스 확률 예측
    rf_test_preds =  rf_classifier.predict( X_valid_imputed )
    ctb_preds = ctb.predict_proba( X_valid )[ : , 1 ]
    ctb_test_preds = ctb.predict_proba( test )[ : , 1 ]

    #  ROC AUC 점수를 계산하여 모델의 성능 평가
    print (roc_auc_score ( y_valid, ctb_preds ) )

    # 메타 트레이닝 및 테스트 예측을 위해
    # CatBoost 모델의 검증 및 테스트 예측을 리스트에 저장
    meta_train = [ ctb_preds, rf_test_preds ]
    meta_test = [ ctb_test_preds ]

    # 최적의 가중치를 찾기 위한 목적 함수를 정의
    def roc_auc( weights ):
        fpred = np.zeros( len( meta_train[ 0 ] ) )
        for i, pred in enumerate( meta_train ):
            fpred += weights[ i ] * pred

        return -roc_auc_score( y_valid, fpred )

    starting_values = [ 0.33 ]*len( meta_train )
    cons = ( { 'type' : 'eq', 'fun' : lambda w : 1 - sum( w ) } )
    bounds = [ ( -1, 1 ) ] * len( meta_train )
    # 최적의 가중치 탐색
    res = minimize( roc_auc,
                    starting_values,
                    method = 'Nelder-Mead',
                    bounds = bounds,
                    constraints = cons )

    print( -res [ "fun" ] )

    # 교차 검증 점수에 최적 가중치에 대한 목적 함수 값의 음수를 더함
    # 최적화 과정에서 최대값 찾음
    cv -= res[ "fun" ]

    # 최적 가중치를 사용하여 테스트 데이터에 대한 예측을 업데이트
    for i, pred in enumerate( meta_test ):
        test_preds += res[ "x" ][ i ] * pred

# 모든 폴드에 대한 교차 검증 점수의 평균을 출력
print( f"CV: { cv / ( fold + 1) }" )

0:	learn: 0.5278203	test: 0.5286454	best: 0.5286454 (0)	total: 23.7ms	remaining: 3m 57s
100:	learn: 0.2847703	test: 0.3504296	best: 0.3504296 (100)	total: 2.44s	remaining: 3m 58s
200:	learn: 0.2388101	test: 0.3425840	best: 0.3425840 (200)	total: 4.87s	remaining: 3m 57s
300:	learn: 0.2065928	test: 0.3395676	best: 0.3392346 (276)	total: 7.29s	remaining: 3m 55s
400:	learn: 0.1802572	test: 0.3358713	best: 0.3358024 (393)	total: 11.6s	remaining: 4m 37s
500:	learn: 0.1597802	test: 0.3327738	best: 0.3324765 (472)	total: 14s	remaining: 4m 26s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.332476499
bestIteration = 472

Shrink model to first 473 iterations.
0.9268155313153995


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9273036416267877
0:	learn: 0.5284940	test: 0.5365441	best: 0.5365441 (0)	total: 23ms	remaining: 3m 49s
100:	learn: 0.2822270	test: 0.3697072	best: 0.3697072 (100)	total: 2.48s	remaining: 4m 2s
200:	learn: 0.2355153	test: 0.3611266	best: 0.3610110 (198)	total: 6.76s	remaining: 5m 29s
300:	learn: 0.2030033	test: 0.3595654	best: 0.3583832 (254)	total: 9.16s	remaining: 4m 55s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3583831871
bestIteration = 254

Shrink model to first 255 iterations.
0.914070377333175


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9141832673009272
0:	learn: 0.5274817	test: 0.5280000	best: 0.5280000 (0)	total: 23.4ms	remaining: 3m 53s
100:	learn: 0.2837244	test: 0.3552520	best: 0.3552520 (100)	total: 2.73s	remaining: 4m 27s
200:	learn: 0.2375239	test: 0.3449787	best: 0.3449787 (200)	total: 6.85s	remaining: 5m 34s
300:	learn: 0.2044085	test: 0.3428425	best: 0.3424989 (286)	total: 9.31s	remaining: 5m
400:	learn: 0.1791655	test: 0.3401399	best: 0.3392037 (375)	total: 11.8s	remaining: 4m 41s
500:	learn: 0.1598086	test: 0.3391675	best: 0.3385316 (471)	total: 14.2s	remaining: 4m 29s
600:	learn: 0.1438787	test: 0.3379921	best: 0.3374967 (560)	total: 17.8s	remaining: 4m 38s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3374966812
bestIteration = 560

Shrink model to first 561 iterations.
0.9260020141297525


...

0.927172616237121
0:	learn: 0.5276304	test: 0.5288368	best: 0.5288368 (0)	total: 23.9ms	remaining: 3m 58s
100:	learn: 0.2813571	test: 0.3505410	best: 0.3505410 (100)	total: 2.43s	remaining: 3m 57s
200:	learn: 0.2356732	test: 0.3414251	best: 0.3411212 (195)	total: 4.82s	remaining: 3m 55s
300:	learn: 0.2031939	test: 0.3384661	best: 0.3379335 (289)	total: 7.49s	remaining: 4m 1s
400:	learn: 0.1786198	test: 0.3366607	best: 0.3366007 (321)	total: 11.6s	remaining: 4m 38s
500:	learn: 0.1582525	test: 0.3359784	best: 0.3353589 (427)	total: 14.1s	remaining: 4m 27s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3353589214
bestIteration = 427

Shrink model to first 428 iterations.
0.9264890798369402


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9267912885522445
0:	learn: 0.5283892	test: 0.5262375	best: 0.5262375 (0)	total: 23.7ms	remaining: 3m 57s
100:	learn: 0.2835621	test: 0.3517189	best: 0.3517189 (100)	total: 2.39s	remaining: 3m 54s
200:	learn: 0.2378814	test: 0.3403691	best: 0.3402981 (187)	total: 6.7s	remaining: 5m 26s
300:	learn: 0.2036161	test: 0.3371053	best: 0.3358636 (274)	total: 9.14s	remaining: 4m 54s
400:	learn: 0.1794047	test: 0.3365293	best: 0.3352178 (319)	total: 11.6s	remaining: 4m 37s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3352178498
bestIteration = 319

Shrink model to first 320 iterations.
0.9240985610761654


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9242821386010313
CV: 5.526248523627125


In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/del_train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/0.5test.csv' )
train.shape, test.shape

((53005, 19), (28068, 19))

In [ ]:
train_     = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test_      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train_.shape, test_.shape, submission.shape, extra_data.shape

((42100, 19), (28068, 18), (28068, 2), (18137, 18))

In [ ]:
# test_preds는 이전 단계에서 각 폴드에서 얻은 예측 결과
# (fold+1)은 폴드의 수로 나누어 결과를 평균화
submission[ "booking_status" ] = test_preds / ( fold + 1 )
y = 'booking_status'

# 테스트 데이터에서 'id' 열을 제외한 중복된 특성 가져옴
dup_features = test_.drop( columns = 'id').columns.tolist( )

# 이너 조인하여 중복된 샘플을 찾음
# 데이터프레임 생성
values_to_assign = test.merge( train_.drop( columns = 'id' ), on = dup_features, how = 'inner' )[[ 'id', y ] ]

# 중복된 샘플의 예측 값을 수정하기 위해 사용할 매핑 딕셔너리를 생성
# booking_status 최대값 : 0, 최소값 : 1 매핑
map_di = { 0 : submission[ y ].max( ), 1 : submission[ y ].min( ) }
# 중복된 샘플의 booking_status 값 수정
# 매핑 딕셔너리를 사용해 수정된 값 할당
submission.loc[ submission.id.isin( values_to_assign.id ), y ] = values_to_assign[ y ].map( map_di ).values

In [ ]:
submission_3 = submission
submission_3

,id,booking_status
0,42100,0.592626
1,42101,0.164090
2,42102,0.918222
3,42103,0.122973
4,42104,0.917848
...,...,...
28063,70163,3.156402
28064,70164,0.106910
28065,70165,0.262539
28066,70166,0.981954


In [ ]:
submission_3.to_csv( "submission_3.csv", index = False )

In [ ]:
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from scipy.optimize import minimize
import numpy as np

In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/del_train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/0.5test.csv' )

In [ ]:
# train.drop( "id", axis = 1, inplace = True )
test.drop( "id", axis = 1, inplace = True )

In [ ]:
train_targets = train[ "booking_status" ]
extra_targets = extra_data[ "booking_status" ]

train.drop(      "booking_status", axis = 1, inplace = True )
extra_data.drop( "booking_status", axis = 1, inplace = True )

In [ ]:
# 각 폴드에 대해 반복
for fold, ( train_idx, valid_idx ) in enumerate( kf.split( train, train_targets ) ):

    #  훈련 데이터와 검증 데이터를 나눔
    X_train, X_valid = train.iloc[ train_idx ], train.iloc[ valid_idx ]
    # 훈련 타겟과 검증 타겟을 나눔
    y_train, y_valid = train_targets.iloc[ train_idx ], train_targets.iloc[ valid_idx ]

    # 추가 데이터를 기존 훈련 데이터에 추가
    X_train = pd.concat( ( X_train, extra_data ) )
    y_train = pd.concat( ( y_train, extra_targets ) )

    # 결측 값 처리를 위한 SimpleImputer
    imputer = SimpleImputer( strategy = 'mean' )
    X_train_imputed = imputer.fit_transform( X_train )
    X_valid_imputed = imputer.fit_transform( X_valid )

    # 랜덤포레스트 모델 정의
    # 랜덤 포레스트 모델 객체 생성
    rf_classifier = RandomForestClassifier( random_state = 42,
                                            n_estimators = 50,
                                            max_depth = 2 )


    # CatBoost 모델 정의
    ctb = CatBoostClassifier( n_estimators = 10000,  # 트리 수 설정
                              early_stopping_rounds = 100,  # 조기 정지 설정
                              **{ # 모델의 하이퍼파라미터 설정
                                  'depth': 2,
                                  'subsample': 0.9037951675853159,
                                  'l2_leaf_reg': 4.085968446512874,
                                  'learning_rate': 0.33153188532829714 } )
    '''
      < subsample >
        - 각 트리가 학습에 사용하는 샘플의 비율을 결정
        - 값이 1보다 작으면 부트스트랩 샘플링 사용
        - 이 값이 클수록 더 많은 다양성을 가진 트리를 생성하며,
          작을수록 더 안정적인 트리를 생성

      < l2_leaf_reg >
        -  L2 정규화의 강도를 결정
        - 과적합을 방지
        - 값이 클수록 정규화의 강도 큼

      < learning_rate >
        - 가중치 업데이트에 대한 학습률
        - 작은 학습률은 모델을 안정적으로 만들지만 오래 걸림
    '''

    # 모델 학습

    # 랜덤포레스트 모델 학습
    rf_classifier.fit( X_train_imputed, y_train )

    # CatBoost 모델 학습
    ctb.fit( X_train, y_train, eval_set = ( X_valid, y_valid ), verbose = 100 )
    # eval_set: 검증 세트를 지정하여 모델의 성능 평가

    # 검증 데이터에 대한 클래스 확률 예측
    rf_test_preds =  rf_classifier.predict( X_valid_imputed )
    ctb_preds = ctb.predict_proba( X_valid )[ : , 1 ]
    ctb_test_preds = ctb.predict_proba( test )[ : , 1 ]

    #  ROC AUC 점수를 계산하여 모델의 성능 평가
    print (roc_auc_score ( y_valid, ctb_preds ) )

    # 메타 트레이닝 및 테스트 예측을 위해
    # CatBoost 모델의 검증 및 테스트 예측을 리스트에 저장
    meta_train = [ ctb_preds, rf_test_preds ]
    meta_test = [ ctb_test_preds ]

    # 최적의 가중치를 찾기 위한 목적 함수를 정의
    def roc_auc( weights ):
        fpred = np.zeros( len( meta_train[ 0 ] ) )
        for i, pred in enumerate( meta_train ):
            fpred += weights[ i ] * pred

        return -roc_auc_score( y_valid, fpred )

    starting_values = [ 0.33 ]*len( meta_train )
    cons = ( { 'type' : 'eq', 'fun' : lambda w : 1 - sum( w ) } )
    bounds = [ ( -1, 1 ) ] * len( meta_train )
    # 최적의 가중치 탐색
    res = minimize( roc_auc,
                    starting_values,
                    method = 'Nelder-Mead',
                    bounds = bounds,
                    constraints = cons )

    print( -res [ "fun" ] )

    # 교차 검증 점수에 최적 가중치에 대한 목적 함수 값의 음수를 더함
    # 최적화 과정에서 최대값 찾음
    cv -= res[ "fun" ]

    # 최적 가중치를 사용하여 테스트 데이터에 대한 예측을 업데이트
    for i, pred in enumerate( meta_test ):
        test_preds += res[ "x" ][ i ] * pred

# 모든 폴드에 대한 교차 검증 점수의 평균을 출력
print( f"CV: { cv / ( fold + 1) }" )

0:	learn: 0.6051140	test: 0.6090835	best: 0.6090835 (0)	total: 65.9ms	remaining: 10m 59s
100:	learn: 0.3815706	test: 0.3908542	best: 0.3908338 (99)	total: 3.42s	remaining: 5m 34s
200:	learn: 0.3668332	test: 0.3804035	best: 0.3804035 (200)	total: 7.61s	remaining: 6m 10s
300:	learn: 0.3576659	test: 0.3745582	best: 0.3745391 (297)	total: 10.8s	remaining: 5m 49s
400:	learn: 0.3522376	test: 0.3691984	best: 0.3691984 (400)	total: 13.9s	remaining: 5m 33s
500:	learn: 0.3470277	test: 0.3663406	best: 0.3663406 (500)	total: 17.3s	remaining: 5m 28s
600:	learn: 0.3429986	test: 0.3632322	best: 0.3632322 (600)	total: 20s	remaining: 5m 12s
700:	learn: 0.3398359	test: 0.3611443	best: 0.3610864 (699)	total: 21.5s	remaining: 4m 44s
800:	learn: 0.3372961	test: 0.3597051	best: 0.3597051 (800)	total: 22.6s	remaining: 4m 19s
900:	learn: 0.3350177	test: 0.3578349	best: 0.3578349 (900)	total: 23.8s	remaining: 3m 59s
1000:	learn: 0.3332292	test: 0.3564704	best: 0.3564399 (998)	total: 24.9s	remaining: 3m 43s
110

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9146430008991099
0:	learn: 0.6050640	test: 0.6097739	best: 0.6097739 (0)	total: 29.2ms	remaining: 4m 51s
100:	learn: 0.3805390	test: 0.4090602	best: 0.4090602 (100)	total: 2.19s	remaining: 3m 34s
200:	learn: 0.3649638	test: 0.3974891	best: 0.3974880 (197)	total: 3.35s	remaining: 2m 43s
300:	learn: 0.3555453	test: 0.3915317	best: 0.3915317 (300)	total: 4.51s	remaining: 2m 25s
400:	learn: 0.3494132	test: 0.3888343	best: 0.3888343 (400)	total: 5.69s	remaining: 2m 16s
500:	learn: 0.3445056	test: 0.3863911	best: 0.3863911 (500)	total: 6.84s	remaining: 2m 9s
600:	learn: 0.3412974	test: 0.3847908	best: 0.3847908 (600)	total: 7.98s	remaining: 2m 4s
700:	learn: 0.3381373	test: 0.3831056	best: 0.3830599 (699)	total: 9.11s	remaining: 2m
800:	learn: 0.3355226	test: 0.3817000	best: 0.3817000 (800)	total: 10.3s	remaining: 1m 57s
900:	learn: 0.3334030	test: 0.3807881	best: 0.3807881 (900)	total: 12s	remaining: 2m 1s
1000:	learn: 0.3310724	test: 0.3795370	best: 0.3795370 (1000)	total: 15.1s	remainin

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9041675213095236
0:	learn: 0.6054189	test: 0.6062213	best: 0.6062213 (0)	total: 10.9ms	remaining: 1m 49s
100:	learn: 0.3807795	test: 0.3887609	best: 0.3887609 (100)	total: 1.17s	remaining: 1m 54s
200:	learn: 0.3659121	test: 0.3795727	best: 0.3795727 (200)	total: 2.31s	remaining: 1m 52s
300:	learn: 0.3574290	test: 0.3734289	best: 0.3734289 (300)	total: 3.66s	remaining: 1m 57s
400:	learn: 0.3514117	test: 0.3699158	best: 0.3699158 (400)	total: 6.23s	remaining: 2m 29s
500:	learn: 0.3468110	test: 0.3673020	best: 0.3672462 (485)	total: 7.86s	remaining: 2m 28s
600:	learn: 0.3431282	test: 0.3652649	best: 0.3652649 (600)	total: 9.13s	remaining: 2m 22s
700:	learn: 0.3393978	test: 0.3633167	best: 0.3633167 (700)	total: 10.3s	remaining: 2m 16s
800:	learn: 0.3367334	test: 0.3620549	best: 0.3619441 (793)	total: 11.4s	remaining: 2m 11s
900:	learn: 0.3343535	test: 0.3612355	best: 0.3612148 (898)	total: 12.6s	remaining: 2m 6s
1000:	learn: 0.3324412	test: 0.3605894	best: 0.3605894 (1000)	total: 13.7s	

...

0.9151545086072654
0:	learn: 0.6051530	test: 0.6094501	best: 0.6094501 (0)	total: 11.3ms	remaining: 1m 52s
100:	learn: 0.3794065	test: 0.3933853	best: 0.3933853 (100)	total: 1.2s	remaining: 1m 57s
200:	learn: 0.3653129	test: 0.3823879	best: 0.3823879 (200)	total: 2.36s	remaining: 1m 55s
300:	learn: 0.3565810	test: 0.3766256	best: 0.3766256 (300)	total: 4.92s	remaining: 2m 38s
400:	learn: 0.3499595	test: 0.3721267	best: 0.3721267 (400)	total: 6.67s	remaining: 2m 39s
500:	learn: 0.3456360	test: 0.3701606	best: 0.3701173 (499)	total: 7.83s	remaining: 2m 28s
600:	learn: 0.3418560	test: 0.3680376	best: 0.3679224 (589)	total: 8.97s	remaining: 2m 20s
700:	learn: 0.3388292	test: 0.3662562	best: 0.3662562 (700)	total: 10.1s	remaining: 2m 14s
800:	learn: 0.3362351	test: 0.3643686	best: 0.3643686 (800)	total: 11.3s	remaining: 2m 9s
900:	learn: 0.3338291	test: 0.3622001	best: 0.3621752 (899)	total: 12.4s	remaining: 2m 5s
1000:	learn: 0.3319447	test: 0.3609641	best: 0.3608981 (989)	total: 13.6s	rem

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9152754241373592
0:	learn: 0.6054822	test: 0.6071379	best: 0.6071379 (0)	total: 12.6ms	remaining: 2m 5s
100:	learn: 0.3810079	test: 0.3886665	best: 0.3886665 (100)	total: 1.19s	remaining: 1m 56s
200:	learn: 0.3655517	test: 0.3783788	best: 0.3783788 (200)	total: 2.34s	remaining: 1m 54s
300:	learn: 0.3569777	test: 0.3738407	best: 0.3738407 (300)	total: 3.49s	remaining: 1m 52s
400:	learn: 0.3504299	test: 0.3697047	best: 0.3696744 (398)	total: 4.64s	remaining: 1m 51s
500:	learn: 0.3458598	test: 0.3671169	best: 0.3671169 (500)	total: 5.8s	remaining: 1m 49s
600:	learn: 0.3424498	test: 0.3652440	best: 0.3652440 (600)	total: 8.16s	remaining: 2m 7s
700:	learn: 0.3396049	test: 0.3634720	best: 0.3634720 (700)	total: 10.1s	remaining: 2m 13s
800:	learn: 0.3367786	test: 0.3617548	best: 0.3617548 (800)	total: 11.2s	remaining: 2m 8s
900:	learn: 0.3345919	test: 0.3606933	best: 0.3605510 (896)	total: 12.4s	remaining: 2m 5s
1000:	learn: 0.3326427	test: 0.3591106	best: 0.3590588 (997)	total: 13.6s	remai

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


0.9146972320674455
CV: 0.9132572487430336


In [ ]:
train      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/del_train.csv' )
test       = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/0.5test.csv' )
train.shape, test.shape

((53005, 19), (28068, 19))

In [ ]:
train_     = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train.csv' )
test_      = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/test.csv' )
submission = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/sample_submission.csv' )
extra_data = pd.read_csv( '/content/drive/MyDrive/멋사_데이터분석스쿨/9.파이널_프로젝트/Binary Classification with a Tabular Reservation Cancellation Dataset/train__dataset.csv' )

train_.shape, test_.shape, submission.shape, extra_data.shape

((42100, 19), (28068, 18), (28068, 2), (18137, 18))

In [ ]:
# test_preds는 이전 단계에서 각 폴드에서 얻은 예측 결과
# (fold+1)은 폴드의 수로 나누어 결과를 평균화
submission[ "booking_status" ] = test_preds / ( fold + 1 )
y = 'booking_status'

# 테스트 데이터에서 'id' 열을 제외한 중복된 특성 가져옴
dup_features = test_.drop( columns = 'id').columns.tolist( )

# 이너 조인하여 중복된 샘플을 찾음
# 데이터프레임 생성
values_to_assign = test.merge( train_.drop( columns = 'id' ), on = dup_features, how = 'inner' )[[ 'id', y ] ]

# 중복된 샘플의 예측 값을 수정하기 위해 사용할 매핑 딕셔너리를 생성
# booking_status 최대값 : 0, 최소값 : 1 매핑
map_di = { 0 : submission[ y ].max( ), 1 : submission[ y ].min( ) }
# 중복된 샘플의 booking_status 값 수정
# 매핑 딕셔너리를 사용해 수정된 값 할당
submission.loc[ submission.id.isin( values_to_assign.id ), y ] = values_to_assign[ y ].map( map_di ).values

In [ ]:
submission_3 = submission
submission_3

,id,booking_status
0,42100,0.077947
1,42101,0.022040
2,42102,0.219945
3,42103,0.029796
4,42104,0.216723
...,...,...
28063,70163,0.613931
28064,70164,0.024131
28065,70165,0.051794
28066,70166,0.312108


In [ ]:
submission_3.to_csv( "submission_3.csv", index = False )

# 결과

1. 원본 데이터
  - RandomForest Depth: 1
  - CatBoost Depth: 3
  - 결과
    - *Private score: 0.91526*: 84 등
    - *Score: 0.92622* : 76 등

2. 중복 변환 데이터
  - 중복 데이더 0, 1 서로 변환
  - RandomForest Depth: 2
  - CatBoost Depth: 2
  - 결과
    - *Private score: 0.91735*: 41 등
    - *Score: 0.92761*: 41 등
3. 중복 제거 데이터
  - RandomForest Depth: 10
  - CatBoost Depth: 8
  - 결과
    - *Private score: 0.91664*: 71 등
    - *Score: 0.92766*: 40 등